<a href="https://colab.research.google.com/github/Tanvir19934/Colab-notebooks/blob/main/GCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.3 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import torch.nn.functional as F


# Define individual costs and locations
individual_costs = torch.tensor([7, 27, 5, 10, 12, 14, 9, 10, 11, 21], dtype=torch.float32)
locations = torch.tensor([
    [0, 0], [1, 2], [2, 3], [3, 4], [4, 5],
    [5, 6], [6, 7], [7, 8], [8, 9], [9, 10]
], dtype=torch.float32)
coalition_cost = torch.tensor(70.0)
# Define edges (for simplicity, assuming a fully connected graph)
edge_index = torch.tensor([
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
     1, 2, 3, 4, 5, 6, 7, 8, 9, 0],
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 0,
     0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
], dtype=torch.long)

# Define the GCN model

class CostSharingGCN(nn.Module):
    def __init__(self, num_node_features, hidden_channels):
        super(CostSharingGCN, self).__init__()
        # Define the three GCN layers
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        # Define the two fully connected layers
        self.fc1 = torch.nn.Linear(hidden_channels, hidden_channels)
        self.fc2 = torch.nn.Linear(hidden_channels, 1)  # Output is one scalar value per node

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # Pass through the first GCN layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        # Pass through the second GCN layer
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        # Pass through the third GCN layer
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        # Pass through the first fully connected layer
        x = self.fc1(x)
        x = F.relu(x)
        # Pass through the second fully connected layer to get the output
        x = self.fc2(x)
        return x
# Prepare data
node_features = torch.cat((individual_costs.view(-1, 1), locations), dim=1)
data = Data(x=node_features, edge_index=edge_index)

# Initialize model, loss function, and optimizer
model = CostSharingGCN(3,16)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(3000):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    payments = model(data).squeeze()
    total_payment = payments.sum()

    # Loss calculation
    budget_balance_loss = criterion(total_payment, torch.tensor(coalition_cost))
    individual_rationality_loss = torch.relu(payments - individual_costs).sum()
    cost_reductions = (individual_costs - payments) / individual_costs
    fairness_loss = cost_reductions.var()

    loss = budget_balance_loss + individual_rationality_loss + fairness_loss*10000

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# Get the final payments
model.eval()
final_payments = model(data).squeeze().detach()
print(f'Final payments: {final_payments}')
print(f'Total payment: {final_payments.sum()}')


<ipython-input-4-3d79c9bbc3e9>:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  budget_balance_loss = criterion(total_payment, torch.tensor(coalition_cost))


Epoch 0, Loss: 1384994.375
Epoch 100, Loss: 999113.125
Epoch 200, Loss: 199128.421875
Epoch 300, Loss: 192271.953125
Epoch 400, Loss: 187642.046875
Epoch 500, Loss: 184392.578125
Epoch 600, Loss: 180627.359375
Epoch 700, Loss: 174974.6875
Epoch 800, Loss: 166703.453125
Epoch 900, Loss: 155825.75
Epoch 1000, Loss: 145117.046875
Epoch 1100, Loss: 140833.28125
Epoch 1200, Loss: 139860.734375
Epoch 1300, Loss: 139202.390625
Epoch 1400, Loss: 138512.984375
Epoch 1500, Loss: 137896.796875
Epoch 1600, Loss: 137382.84375
Epoch 1700, Loss: 136859.6875
Epoch 1800, Loss: 136320.28125
Epoch 1900, Loss: 135766.9375
Epoch 2000, Loss: 135200.265625
Epoch 2100, Loss: 134644.125
Epoch 2200, Loss: 134060.484375
Epoch 2300, Loss: 133484.875
Epoch 2400, Loss: 132924.921875
Epoch 2500, Loss: 132435.90625
Epoch 2600, Loss: 130913.5078125
Epoch 2700, Loss: 127669.0234375
Epoch 2800, Loss: 123247.1328125
Epoch 2900, Loss: 117658.078125
Final payments: tensor([ 86.1148,  57.3928,  58.6434, 112.1279, 134.9170, 

In [ ]:
(individual_costs-final_payments)/individual_costs

tensor([0.4291, 0.4233, 0.4301, 0.4332, 0.3481, 0.5298, 0.4240, 0.4393, 0.3771,
        0.5358])

In [ ]:
individual_costs

tensor([ 7., 27.,  5., 10., 12., 14.,  9., 10., 11., 21.])

In [ ]:
coalition_cost

tensor(0.6431, grad_fn=<SumBackward0>)

In [ ]:
torch.tensor([70,70,70,70,70,70,70,70,70,70])/10

tensor([7., 7., 7., 7., 7., 7., 7., 7., 7., 7.])

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.fc1 = torch.nn.Linear(hidden_channels, hidden_channels)
        self.fc2 = torch.nn.Linear(hidden_channels, 1)  # Output is one scalar value per node

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

# Example data with provided individual costs
num_nodes = 10
features = 2  # For example, location and individual cost
x = torch.randn((num_nodes, features))
edge_index = torch.tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                           [1, 2, 3, 4, 5, 6, 7, 8, 9, 0]], dtype=torch.long)
individual_costs = torch.tensor([7, 27, 5, 10, 12, 14, 9, 10, 11, 21], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)

# Initialize model, optimizer, and loss function
model = GCN(num_node_features=features, hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

# Define the target coalition cost
target_coalition_cost = torch.tensor(70.0)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    out = model(data).squeeze()
    coalition_cost = out.sum()
    payments = out

    # Calculate percentage reduction in costs
    percentage_reductions = (individual_costs - payments) / individual_costs

    # Calculate the loss
    loss = criterion(coalition_cost, target_coalition_cost)  # Ensure budget balance
    individual_rationality_loss = torch.mean(F.relu(payments - individual_costs))  # Penalty for exceeding individual cost
    fairness_loss = torch.var(percentage_reductions)*10000  # Penalize large variations in percentage reduction
    loss += individual_rationality_loss + fairness_loss  # Ensure individual rationality and fairness

    # Backpropagation and optimization
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# Resulting payments
model.eval()
with torch.no_grad():
    payments = model(data).squeeze()
    print("Payments:", payments)
    print("Total Payments:", payments.sum().item())


Epoch 0, Loss: 4831.94873046875
Epoch 100, Loss: 556.5801391601562
Epoch 200, Loss: 501.48291015625
Epoch 300, Loss: 393.09613037109375
Epoch 400, Loss: 345.3827209472656
Epoch 500, Loss: 271.55169677734375
Epoch 600, Loss: 0.019259866327047348
Epoch 700, Loss: 6.775638894396252e-07
Epoch 800, Loss: 0.011898737400770187
Epoch 900, Loss: 6.570100907765664e-08
Payments: tensor([ 3.8506, 14.9894,  2.7730,  5.5480,  6.6560,  7.7633,  4.9699,  5.5204,
         6.0992, 11.6257])
Total Payments: 69.79550170898438


In [ ]:
individual_costs

tensor([ 7., 27.,  5., 10., 12., 14.,  9., 10., 11., 21.])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define individual costs and coalition cost
individual_costs = torch.tensor([7, 27, 5, 10, 12, 14, 9, 10, 11, 21], dtype=torch.float32)
coalition_cost = 70.0

# Define the model
class CostSharingNN(nn.Module):
    def __init__(self, input_dim):
        super(CostSharingNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model, loss function, and optimizer
model = CostSharingNN(input_dim=1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(1000):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    payments = model(individual_costs.view(-1, 1)).squeeze()
    total_payment = payments.sum()

    # Loss calculation
    budget_balance_loss = criterion(total_payment, torch.tensor(coalition_cost))
    individual_rationality_loss = torch.relu(payments - individual_costs).sum()
    cost_reductions = (individual_costs - payments) / individual_costs
    fairness_loss = cost_reductions.var()

    loss = budget_balance_loss + individual_rationality_loss + fairness_loss

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# Get the final payments
model.eval()
final_payments = model(individual_costs.view(-1, 1)).squeeze().detach()
print(f'Final payments: {final_payments}')
print(f'Total payment: {final_payments.sum()}')


Epoch 0, Loss: 6528.228515625
Epoch 100, Loss: 0.013233574107289314
Epoch 200, Loss: 0.0016287502367049456
Epoch 300, Loss: 0.0016283998265862465
Epoch 400, Loss: 0.0016279863193631172
Epoch 500, Loss: 0.001627505524083972
Epoch 600, Loss: 0.0016269544139504433
Epoch 700, Loss: 0.0016263427678495646
Epoch 800, Loss: 0.0016256696544587612
Epoch 900, Loss: 0.0016249326290562749
Final payments: tensor([ 4.2690, 14.0355,  3.2992,  5.7266,  6.7041,  7.6816,  5.2389,  5.7266,
         6.2154, 11.1030])
Total payment: 69.99998474121094


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.2 MB/s eta 0:00:00


In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 947.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.2 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import torch.nn.functional as F
import itertools

# Define individual costs and locations
individual_costs = torch.tensor([7, 27, 5, 10, 12, 14, 9, 10, 11, 21], dtype=torch.float32)
locations = torch.tensor([
    [0, 0], [1, 2], [2, 3], [3, 4], [4, 5],
    [5, 6], [6, 7], [7, 8], [8, 9], [9, 10]
], dtype=torch.float32)
coalition_cost = torch.tensor(70.0)
# Define edges (for simplicity, assuming a fully connected graph)
edge_index = torch.tensor([
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
     1, 2, 3, 4, 5, 6, 7, 8, 9, 0],
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 0,
     0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
], dtype=torch.long)

# Define the GCN model
class CostSharingGCN(nn.Module):
    def __init__(self, num_node_features, hidden_channels):
        super(CostSharingGCN, self).__init__()
        # Define the three GCN layers
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        # Define the two fully connected layers
        self.fc1 = torch.nn.Linear(hidden_channels, hidden_channels)
        self.fc2 = torch.nn.Linear(hidden_channels, 1)  # Output is one scalar value per node

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # Pass through the first GCN layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        # Pass through the second GCN layer
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        # Pass through the third GCN layer
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        # Pass through the first fully connected layer
        x = self.fc1(x)
        x = F.relu(x)
        # Pass through the second fully connected layer to get the output
        x = self.fc2(x)
        return x

# Prepare data
node_features = torch.cat((individual_costs.view(-1, 1), locations), dim=1)
data = Data(x=node_features, edge_index=edge_index)

# Initialize model, loss function, and optimizer
model = CostSharingGCN(3, 32)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define stability loss function
def stability_loss(payments, individual_costs):
    num_nodes = len(individual_costs)
    loss = torch.tensor(0.0)
    for i in range(1, num_nodes):
        for subset in itertools.combinations(range(num_nodes), i):
            subset_payment = payments[list(subset)].sum()
            subset_cost = individual_costs[list(subset)].sum()
            if subset_payment > subset_cost:
                loss += subset_payment - subset_cost
    return loss

# Training loop
for epoch in range(2000):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    payments = model(data).squeeze()
    total_payment = payments.sum()

    # Loss calculation
    budget_balance_loss = criterion(total_payment, torch.tensor(coalition_cost))
    individual_rationality_loss = torch.relu(payments - individual_costs).sum()
    cost_reductions = (individual_costs - payments) / individual_costs
    fairness_loss = cost_reductions.var()
    stability_loss_value = stability_loss(payments, individual_costs)

    loss = budget_balance_loss + individual_rationality_loss + fairness_loss * 10000 + stability_loss_value

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# Get the final payments
model.eval()
final_payments = model(data).squeeze().detach()
print(f'Final payments: {final_payments}')
print(f'Total payment: {final_payments.sum()}')


<ipython-input-2-12653e23c069>:85: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  budget_balance_loss = criterion(total_payment, torch.tensor(coalition_cost))


Epoch 0, Loss: 4292.814453125
Epoch 100, Loss: 699.85791015625
Epoch 200, Loss: 629.7369995117188
Epoch 300, Loss: 604.4925537109375
Epoch 400, Loss: 555.0797119140625
Epoch 500, Loss: 389.4798278808594
Epoch 600, Loss: 337.3009033203125
Epoch 700, Loss: 301.0767822265625
Epoch 800, Loss: 223.76576232910156
Epoch 900, Loss: 104.62883758544922
Epoch 1000, Loss: 27.746665954589844
Epoch 1100, Loss: 10.290665626525879
Epoch 1200, Loss: 8.5853271484375
Epoch 1300, Loss: 7.433615684509277
Epoch 1400, Loss: 6.9077630043029785
Epoch 1500, Loss: 6.428675651550293
Epoch 1600, Loss: 5.986484527587891
Epoch 1700, Loss: 5.479331016540527
Epoch 1800, Loss: 5.265585422515869
Epoch 1900, Loss: 4.722656726837158
Final payments: tensor([ 3.8994, 15.0459,  2.7822,  5.5673,  6.6793,  7.5713,  5.4402,  5.2467,
         6.1777, 11.7035])
Total payment: 70.11363983154297


In [ ]:
stability_loss(payments, individual_costs)

tensor(0.)

In [ ]:
loss

tensor(0.0974, grad_fn=<AddBackward0>)